<a href="https://colab.research.google.com/github/rashida048/ApacheSpark-BigDataAnalytics/blob/main/Assignment3_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [ ]:
rd = sc.textFile("taxi-data-sorted-small.csv.bz2")

In [ ]:
words = rd.map(lambda x: x.split(','))

In [ ]:
def isfloat(value):
    try:
        float(value)
        return True
    except:
        return False

In [ ]:
def correctRows(p):
    if(len(p)==17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[5]) !=0 and float(p[11]) !=0):
                return p

In [ ]:
wordsCorr = words.filter(lambda x: correctRows(x))

In [ ]:
wordsCorr.take(2)

[['07290D3599E7A0D62097A346EFCC1FB5',
  'E7750A37CAB07D0DFF0AF7E3573AC141',
  '2013-01-01 00:00:00',
  '2013-01-01 00:02:00',
  '120',
  '0.44',
  '-73.956528',
  '40.716976',
  '-73.962440',
  '40.715008',
  'CSH',
  '3.50',
  '0.50',
  '0.50',
  '0.00',
  '0.00',
  '4.50'],
 ['0EC22AAF491A8BD91F279350C2B010FD',
  '778C92B26AE78A9EBDF96B49C67E4007',
  '2013-01-01 00:01:00',
  '2013-01-01 00:03:00',
  '120',
  '0.71',
  '-73.973145',
  '40.752827',
  '-73.965897',
  '40.760445',
  'CSH',
  '4.00',
  '0.50',
  '0.50',
  '0.00',
  '0.00',
  '5.00']]

In [ ]:
def zeroOrOne(x):
  if (x >= 1 and x <= 6):
    return 1.0
  else:
    return 0.0

In [ ]:
night_ride_add = wordsCorr.map(lambda x: x + [zeroOrOne(int(x[1][-7]))])

In [ ]:
five_feat = night_ride_add.map(lambda x: (x[1], float(x[4])/3600, float(x[5]), 1.0, float(x[15]), x[-1], float(x[11]) + float(x[14]) ))

In [ ]:
five_feat.take(3)

[('E7750A37CAB07D0DFF0AF7E3573AC141',
  0.03333333333333333,
  0.44,
  1.0,
  0.0,
  0.0,
  3.5),
 ('778C92B26AE78A9EBDF96B49C67E4007',
  0.03333333333333333,
  0.71,
  1.0,
  0.0,
  1.0,
  4.0),
 ('BE317B986700F63C43438482792C8654',
  0.03333333333333333,
  0.48,
  1.0,
  0.0,
  0.0,
  4.0)]

In [ ]:
five_feat_arr = five_feat.groupByKey()

In [ ]:
aggr = five_feat.groupByKey().mapValues(list)